### 7. 데이터 합치기와 재구성

#### 7.1 멀티 인덱스

In [2]:
import numpy as np
import pandas as pd

Series의 멀티 인덱스

In [ ]:
# 학생 점수 데이터를 시리즈로 생성
# 값: 점수
# 인덱스는 2단계
# 첫번째 레벨: 학년 (1학년/2학년)
# 두번째 레벨: 빈 (A반/B반)
s = pd.Series(
    [85, 90, 78, 92, 88, 75],
    index=[["1학년", "1학년", "1학년", "2학년", "2학년", "2학년"],
           ["a반", "a반", "b반", "a반", "b반", "b반"]]
)

인덱스로 값 꺼내기

In [ ]:
# 1학년만 조회
s['1학년']

# 1학년 a반만 조회
# 멀티 인덱스는 튜플을 사용
s.loc[('1학년','a반')]

# 1학년에서 여러반 선택
s.loc[('1학년',['a반','b반'])]

DataFrame의 인덱스

In [3]:
# 매출 데이터
df = pd.DataFrame({
    "날짜": ["2025-01-01", "2025-01-02", "2025-01-03"],
    "매출": [100, 130, 90]
})

# 날짜를 인덱스로 설정
df = df.set_index("날짜")
print(df)

# 인덱스를 다시 열로 되돌리기
df = df.reset_index()
print(df)

             매출
날짜             
2024-01-01  100
2024-01-02  130
2024-01-03   90
           날짜   매출
0  2024-01-01  100
1  2024-01-02  130
2  2024-01-03   90


DataFrame의 멀티 인덱스

In [ ]:
# 상품 판매량 데이터를 데이프레임으로 생성
# 값: 상품 판매량
# 첫번째 인덱스: 연도
# 두번째 인덱스: 지점
# 컬럼: 제품명
df = pd.DataFrame([[50, 60],[70, 80],[100, 110],[120, 130]],
                  index=[["2020년","2020년","2021년","2021년"],
                         ["구월점","부평점","구월점","부평점"]],
                  columns=["제품1","제품2"])
df

인덱스로 값 꺼내기

In [ ]:
# 2020년 전체 지점 조회하기
df.loc["2020년"]

# 2020년 구월점만 조회하기
df.loc[("2020년", "구월점")]

# 2021년 부평점의 제품2 판매량만 조회하기
df.loc[("2021년", "부평점"), "제품2"]

#### 7.2 데이터 합치기

merge
- 두개의 데이터 프레임을 합치는 함수
- 기준 컬럼이 맞으면 서로 연결

In [ ]:
# 학생 데이터
df1 = pd.DataFrame({
    "학번": [1, 2, 3],
    "이름": ["철수", "영희", "민수"]
})

# 시험 점수 데이터
df2 = pd.DataFrame({
    "학번": [1, 2, 4],
    "점수": [90, 85, 70]
})
df1

merge의 종류
- inner join: 양쪽 데이터에서 겹치는 값만 가지고옴
- left join: 왼쪽 데이터는 모두 가져오고, 오른쪽은 겹치는 부분이 없으면 nan
- right join: 오른쪽 데이터는 모두 가져오고, 왼쪽은 겹치는 부분이 없으면 nan
- outer join: 양쪽 데이터를 모두 가져오기

In [ ]:
# 두 데이터 합치기 (공통 컬럼은 학번)

# 두 테이터에서 공통된 학번만 가져오기
pd.merge(df1, df2, on="학번")

# # 왼쪽 데이터는 모두 가져오고, 오른쪽은 겹치는 부분이 없으면 x
# pd.merge(df1, df2, on="학번", how="left")

# # 오른쪽 데이터는 모두 가져오고, 오른쪽은 겹치는 부분이 없으면 x
# pd.merge(df1, df2, on="학번", how="right")

# # 양쪽 데이터를 모두 가져오기
pd.merge(df1, df2, on="학번", how="outer")

concat
- 여러개의 데이터 프레임(또는 시리즈)을 이어 붙이는 함수
- merge 처럼 조건으로 맞추는게 아니라, 그냥 이어 붙이는 것
- 방향: 위아래 또는 옆으로

In [ ]:
# 기본값 axis=0 -> 위아래로 이어 붙이기
# 단순히 이어붙이는 거라 인덱스가 중복됨
pd.concat([df1, df2])  

# 새 인덱스로 재설정
# pd.concat([df1, df2], ignore_index=True)

In [ ]:
# 기본값 axis=1 -> 옆으로 이어 붙이기
pd.concat([df1, df2], axis=1)

#### 7.3 데이터 재구성

피벗(pivot)
- 행열을 바꿔서 표를 재구성하는 함수

In [ ]:
df = pd.DataFrame({
    "이름": ["철수", "영희", "철수", "영희"],
    "과목": ["국어", "국어", "수학", "수학"],
    "점수": [90, 85, 80, 95]
})
df

# 컬럼 안에 있는 값을 인덱스와 컬럼으로 사용
# 표 구조를 재구성함
result = df.pivot(index="이름", columns="과목", values="점수")
# result

# # 행열 확인
# print(result.index)
# print(result.columns)